# TiXI Tutorial: Read and write CPACS files

In this tutorial we will learn how to:
- initialize the TiXI library
- read data from a CPACS file
- Write new data to a new CPACS file

Here are some helpful resources for [TiXI](https://github.com/DLR-SC/tixi):
- Binary Downloads:  https://github.com/DLR-SC/tixi/wiki/Downloads
- API Documentation: https://dlr-sc.github.io/tixi/
- Issue Tracker:     https://github.com/DLR-SC/tixi/issues
- Wiki:              https://github.com/DLR-SC/tixi/wiki

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", None)
import matplotlib.pyplot as plt

## Initialize TiXI

The first thing we need to do is import and initialise TiXI.

In [ ]:
from tixi3 import tixi3wrapper
tixi_h = tixi3wrapper.Tixi3()

Let's examine this object to see what functionality it provides, using the Python help method [Python help method](https://docs.python.org/3/library/functions.html#help):

## Open a CPACS File

In this tutorial, we will be using an aircraft model from the [Digital Hangar](https://www.digital-hangar.de/). The Digital Hangar provides aircraft designs in the CPACS data format to the research community. 

Download the `EXACT Turbofan Baseline` and drag it into this working directory to open it with TiXI. Alternatively, it is available in the `resources` folder.

In [ ]:
tixi_h.open('D250-TF-2040.xml')

You can also go to that directory and open the file to explore its contents.
It contains some basic description of an aircraft model and analysis results. In this example, we want to access the `Analyses` Node to read a `Trajectory` of the Aircraft

## Validate the dataset

In [ ]:
try:
    tixi_h.schemaValidateFromFile('resources/cpacs_schema.xsd')
    print("✅ Schema validation successful.")
except tixi3wrapper.Tixi3Exception as e:
    print("❌ Schema validation failed!")
    print(f"   Reason: {e}")

## Read Data from the CPACS Model

<img src="resources/dataStructure.png" style="width:80%;"/>

### Get string elements

First we are going to read some data from the model. Reading is always done using one of the `tixi_h.get...()` methods which take an [XPath](https://en.wikipedia.org/wiki/XPath) as their first argument. This `XPath` is basically a query that identifies `Elements` or `Attributes` within a XML file.

Let us get the `name` of the dataset using the path to the header and selecting it via its index:
```xml
<?xml version="1.0"?>
<cpacs xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.cpacs.de/schema/v3_5_0/cpacs_schema.xsd">
  <header>
    <name>D250-TF-2040_orig</name>
    <version>1.0</version>
    <cpacsVersion>3.5</cpacsVersion>
    <versionInfos>
      <versionInfo version="0.1">
        <description>Original data set provided by the DLR Exact project</description>
        <creator>openAD</creator>
        <timestamp>2023-12-18T13:22:16</timestamp>
        <cpacsVersion>3.2</cpacsVersion>
      </versionInfo>
      <versionInfo version="1.0">
        <description>Data set updated to cpacsVersion 3.5</description>
        <creator>DLR-SL</creator>
        <timestamp>2024-02-07T15:06:12Z</timestamp>
        <cpacsVersion>3.5</cpacsVersion>
      </versionInfo>
    </versionInfos>
  </header>

```

In [ ]:
dataset_name = tixi_h.getTextElement("/cpacs/header/name")
print(f"Dataset name: {dataset_name}")

### Fun with XPath

In our data we find one aircraft model:
```xml
<?xml version="1.0"?>
<cpacs xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.cpacs.de/schema/v3_5_0/cpacs_schema.xsd">
  <header>
    ...
  </header>
  <vehicles> 
    ...
    <aircraft>
      <model uID="AircraftModel">
        <name>EXACT Turbofan</name>
        <description>openAD</description>
```
So the simplest way to define the XPath would be:

In [ ]:
aircraft_xpath = "/cpacs/vehicles/aircraft/model"
tixi_h.checkElement(aircraft_xpath)

We could also use the `uID` attribute to get the XPath automatically via TiXI:

In [ ]:
aircraft_xpath = tixi_h.uIDGetXPath('AircraftModel')
tixi_h.checkElement(aircraft_xpath)

### Reading trajectories
```xml
<cpacs xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.cpacs.de/schema/v3_5_0/cpacs_schema.xsd">
  <vehicles>
    <aircraft>
      <model uID="AircraftModel">
        <analyses>
          <trajectories>
            <trajectory uID="block_150nm_23750kg">
              <name>Block 150nm 23750kg</name>
              ...
            </trajectory>
            <trajectory uID="reserve_150nm_23750kg">
              <name>Reserve 150nm 23750kg</name>
            </trajectory>
            ...
          </trajectories>
        </analyses>
      </model>
    </aircraft>
  </vehicles>
</cpacs>
```

In [ ]:
trajectories_xpath = f"{aircraft_xpath}/analyses/trajectories"
n_traj = tixi_h.getNamedChildrenCount(trajectories_xpath, "trajectory")
print(f"Number of trajectories: {n_traj}")

In [ ]:
traj_info = []
for i in range(1, n_traj + 1):
    trajectory_xpath = f"{trajectories_xpath}/trajectory[{i}]"
    name = tixi_h.getTextElement(f"{trajectory_xpath}/name") if tixi_h.checkElement(f"{trajectory_xpath}/name") else "N/A"
    uid = tixi_h.getTextAttribute(trajectory_xpath, "uID") if tixi_h.checkAttribute(trajectory_xpath, "uID") else "N/A"
    traj_info.append({"Index": i, "uID": uid, "Name": name, "xPath": trajectory_xpath})
pd.DataFrame(traj_info)

### Processing a single trajectory

For the tutorial, we take a look at the **first** trajectories of the aircraft. Each trajectory consists of `flightPoints`: comma-separated vectors to store large data:
```xml
<cpacs xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.cpacs.de/schema/v3_5_0/cpacs_schema.xsd">
  <vehicles>
    <aircraft>
      <model uID="AircraftModel">
        <analyses>
          <trajectories>
            <trajectory uID="block_150nm_23750kg">
              <name>Block 150nm 23750kg</name>
              <global>
                <fuelMass>1397.57</fuelMass>
                ...
              </global>
              <flightPoints>
                <index>0;1;2;3;4;5;...</index>
                <segmentUID>taxi_out;taxi_out;take_off;climb_cas_mach_01_design;climb_cas_mach_01_design;...</segmentUID>
                <flightTime>0;540;660;667;674;...</flightTime>
                <flightDistance>0;0;0;919.873;1842.72;...</flightDistance>
                <altitude>0;0;457.2;526.095;594.673;...</altitude>
                ...
              </flightPoints>
            </trajectory>
```

In [ ]:
trajectory_xpath = f"{trajectories_xpath}/trajectory[1]"
flightPoints_xpath = f"{trajectory_xpath}/flightPoints"

In the first step, we want to find out how many elements the trajectory vector has. Therefore, we get the vector size of the `index` vector with the tixi function `getVectorSize`.

In [ ]:
vector_size = tixi_h.getVectorSize(f"{flightPoints_xpath}/index")
print(f"Each vector is of our trajectory has {vector_size} entries.")

In the next step, we want to get the `time` vector, as well as the `altitude` vector of our trajectory. We use this opportunity to introduce two methods of accessing the data:

(1) Get vector as splitted string:

In [ ]:
time_vector_str = tixi_h.getTextElement(f"{flightPoints_xpath}/flightTime").split(';')
time_vector = [float(t) for t in time_vector_str]
print(time_vector)

(2) Obtain the vector using the Tixi convenience function `getFloatVector`:

In [ ]:
altitude_vector = tixi_h.getFloatVector(f"{flightPoints_xpath}/altitude", vector_size)
print(altitude_vector)

Now we want to list the vectors in a table.

In [ ]:
df = pd.DataFrame({
    "Time [s]": time_vector,
    "Altitude [m]": altitude_vector
})
df

We could plot the trajectory via [Matplotlib](https://matplotlib.org/):

In [ ]:
trajectory_name = tixi_h.getTextElement(f"{trajectory_xpath}/name") if tixi_h.checkElement(f"{trajectory_xpath}/name") else "N/A"

plt.figure(figsize=(8, 5))
plt.plot(time_vector, altitude_vector, marker="o", linestyle="-", linewidth=2, markersize=6, label="Altitude [m]")

plt.xlabel("Time [s]", fontsize=12)
plt.ylabel("Altitude [m]", fontsize=12)
plt.title(f"Aircraft trajectory {trajectory_name}", fontsize=14, fontweight="bold")

plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()

plt.tight_layout()
plt.show()

## Write Data to CPACS File

We now want to add a new trajectory to the CPACS dataset.
For simplicity, the trajectory vectors consist of just seven points:

In [ ]:
altitude = [0,8000,8000,11000,11000,7000,0]
time = [0,500,2000,3000,8000,9000,11000]
plt.plot(time, altitude, marker='o')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (m)')
plt.title('New Trajectory Profile')
plt.show()

We need to understand the schema to set the correct data structure. For example, we can you [XSDDiagram](https://www.cpacs.de/3rdParty/XSDDiagram.zip) or the [CPACS online documentation](https://dlr-sl.github.io/cpacs-website/documentation/CPACS_3_5_0_Docs/html/7bb51e80-aa8b-2d94-eade-4d7c33513a09.htm):

<img src="resources/trajectory_xsd.png" style="width:60%;"/>

First, we create a new `trajectory` element with the `uID` `Our_CPACS_Journey` in the `trajectories` node:

In [ ]:
tixi_h.createElement(trajectories_xpath,'trajectory')
newTrajectory_xpath = f"{trajectories_xpath}/trajectory[last()]"

uID = "Our_CPACS_journey"
name = "Our CPACS journey"
tixi_h.addTextAttribute(newTrajectory_xpath,'uID', uID)
tixi_h.addTextElement(newTrajectory_xpath, 'name', name)

Then we add a float vector to the new trajectory. The length of the vector and the number of digits after the comma must be specified.

In [ ]:
tixi_h.createElement(newTrajectory_xpath, "flightPoints")

new_flightPoints_xpath = f"{newTrajectory_xpath}/flightPoints"
tixi_h.addFloatVector(new_flightPoints_xpath, "flightTime", time, numElements=len(altitude), format="%.1f")
tixi_h.addFloatVector(new_flightPoints_xpath, "altitude", altitude, numElements=len(altitude), format="%.1f")

Next, we want to check the new information. Therefore, we introduce a different possibility to get the trajectory information. We can get the xPath of the trajectory from it's uID.

In [ ]:
check_new_xpath = tixi_h.uIDGetXPath(uID)
print(f"A trajectory with uID `Our_CPACS_Journey` is found at {check_new_xpath}.")

### Validate the data

In [ ]:
try:
    tixi_h.schemaValidateFromFile('resources/cpacs_schema.xsd')
    print("✅ Schema validation successful.")
except tixi3wrapper.Tixi3Exception as e:
    print("❌ Schema validation failed!")
    print(f"   Reason: {e}")

Unfortunately, TiXI does not show us the source of the error here, but a look at the schema reveals that we still need the elements `index`, `segmentUID`, `thrust`, `latitude` and `longitude`.

In [ ]:
index_vec = ";".join([str(i) for i in range(len(altitude))])
tixi_h.addTextElementAtIndex(new_flightPoints_xpath, "index", index_vec, 1)

In [ ]:
segment_uids = ";".join(["take_off","cruise_const_altitude"*4,"approach_landing_design", "taxi_in"])
tixi_h.addTextElementAtIndex(new_flightPoints_xpath, "segmentUID", segment_uids, 2)

As the other data is unavailable, we use `NaN` vectors to indicate this:

In [ ]:
NaN_vec = ";".join(["NaN" for i in range(len(altitude))])
tixi_h.addTextElementAtIndex(new_flightPoints_xpath, "thrust", NaN_vec, 4)
tixi_h.addTextElementAtIndex(new_flightPoints_xpath, "latitude", NaN_vec, 5)
tixi_h.addTextElementAtIndex(new_flightPoints_xpath, "longitude", NaN_vec, 6)

In [ ]:
try:
    tixi_h.schemaValidateFromFile('resources/cpacs_schema.xsd')
    print("✅ Schema validation successful.")
except tixi3wrapper.Tixi3Exception as e:
    print("❌ Schema validation failed!")
    print(f"   Reason: {e}")

## Some comments on the CPACS development

Congratulations, you have just found an aspect of CPACS that should be improved. Fortunately, CPACS development has not stopped after 20 years, and in a current research project with an intensive focus on trajectories, we have developed a modified trajectory definition that makes the mandatory elements optional, removes the mandatory order, adds new elements and presents the syntax in a more modern way. This will be part of the CPACS v3.6 release:

<img src="resources/trajectory_xsd_new.png" style="width:60%;"/>

## Save a CPACS File

Once we have modified the CPACS model using TiXI, we can save the updated dataset as a new XML file:

In [ ]:
tixi_h.save('D250-TF-2040_NEW.xml')
tixi_h.close()

Don't forget to call the `tixi_h.close()` method after saving!

You can now also open that CPACS file with a text editor and see your modifications.